# Lab 2 : Classification

## G3 SDI - Machine Learning

In this lab, we will tackle a binary classification problem. We are going to use the MNIST dataset, which contains 28x28 images of handwritten digits. More precisely, our goal will be to distinguish between 3s and 5s using logistic regression and K-nearest neighbors.

### Instructions
* Rename your notebook with your surnames as `lab2_Name1_Name2.ipynb`, and include your names in the notebook.
* Your code, and its output, must be commented !
* Please upload your notebook on Moodle in the dedicated section before the deadline.

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Report written by [name1], [name2], date.
</div>

In [ ]:
# Import usual libraries
import numpy as np
from matplotlib import pyplot as plt

**Q1.** Load the data from the `.npy` files included in the archive (use `np.load`). How many examples are there ? How many features ? What do the values of the features represent ?

In [ ]:
##########
## YOUR CODE HERE
##########

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Your answer here
</div>

**Q2.** Display a few examples of 3, and a few examples of 5 (use `plt.imshow`). Comment.

In [ ]:
##########
## YOUR CODE HERE
##########

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Your answer here
</div>

**Q3.** Create a variable X which contains all the images (use `np.vstack`), and a variable y which encodes the class.

Normalize the features between 0 and 1.

In [ ]:
##########
## YOUR CODE HERE
##########

**Q4.** Split the dataset into a training set and a test set, using 20% of the original dataset for the test set. Do not forget to set the random state.

In [ ]:
from sklearn.model_selection import train_test_split

##########
## YOUR CODE HERE
##########

**Q5.** Train a logistic regression model on the training set (see documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)). Recall how the coefficients are obtained.

What does scikit-learn do by default that does not match what we studied during the lecture ?

In [ ]:
from sklearn.linear_model import LogisticRegression

##########
## YOUR CODE HERE
##########

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Your answer here
</div>

**Q6.** Compute the accuracy on the test set.

In [ ]:
##########
## YOUR CODE HERE
##########

**Q7.** Display the confusion matrix from the test set (add label names), and interpret it.

In [ ]:
##########
## YOUR CODE HERE
##########

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Your answer here
</div>

**Q8.** Display an example where a 3 was mistaken for 5, and vice versa. Comment.

In [ ]:
##########
## YOUR CODE HERE
##########

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Your answer here
</div>

**Q9.** As we have seen during the lecture, the logistic regression model does not directly output a 0 or a 1, but rather the probability of being 0 or 1 (given x), which can be accessed with the `.predict_proba` attribute. This probability is then compared to a threshold to output the prediction, the default threshold being 0.5.

Recall what is the underlying assumption of using a threshold of 0.5.

Display the evolution of the precision and recall (using the 5s as the reference class), as well as the F1-score, when the threshold varies between 0 and 1. Comment.

In [ ]:
##########
## YOUR CODE HERE
##########

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Your answer here
</div>

**Q10.** Lastly, we would like to check the performance of KNN on this dataset. To do so, select the optimal value of $K$ by cross-validation on the training set, and then compute the accuracy score of the optimal KNN on the test set. Comment.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

##########
## YOUR CODE HERE
##########

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Your answer here
</div>